### Getting started with Backtrader
This is part of the KE5207 project assignment.

In [ ]:
from os import listdir
from os.path import isfile, join
def getFilesAndNames(dirpath):
    ''' Return only the Futures files from the directory'''
    onlyfiles = []
    for f in listdir(dirpath):
        fullpath = join(dirpath, f)
        if isfile(fullpath) and f.startswith("F"):
            onlyfiles.append((fullpath,f.split(".")[0]))
    #print (onlyfiles)
    return onlyfiles

#getFilesAndNames(os.getcwd() + "\\Data\\")

### Setup the trading rules and platform here

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

from __future__ import (absolute_import, division, print_function,
                        unicode_literals)
import datetime  # For datetime objects
import os.path  # To manage paths
import sys  # To find out the script name (in argv[0])

# Import the backtrader platform
import backtrader as bt
import pandas as pd

In [ ]:
modpath = os.getcwd() + "\\Data\\Prod\\" #os.path.dirname(os.path.abspath(sys.argv[0]))
file = 'FCPO0114.csv'
datapath = os.path.join(modpath, file)
df = pd.read_csv(datapath)
df.head()
#df['DateTime'][0]
print (df['DateTime'][len(df)-1])
obj = datetime.datetime(2011, 1, 3)
print (type(obj))
dtobj = datetime.datetime.strptime(df['DateTime'][len(df)-1], "%d/%m/%Y %H:%M")
print (type(dtobj))

In [ ]:
import pytz

# setup data
def setup_multiple_data(bt, cerebro):
    import os
    modpath = os.getcwd() + "\\Data\\Test\\"
    datalist = getFilesAndNames(modpath)
    
    
    #start_test = datetime.datetime(2011, 1, 3)
    #end_test = datetime.datetime(2014, 1, 1)
    #start_trade = datetime.datetime(2014, 1, 2)
    #end_trade = datetime.datetime(2016, 12, 30)
    
    for i in range(len(datalist)):
        df = pd.read_csv(datalist[i][0])
        start_test = datetime.datetime.strptime(df['DateTime'][0], "%d/%m/%Y %H:%M")
        end_test = datetime.datetime.strptime(df['DateTime'][len(df)-1], "%d/%m/%Y %H:%M")
        
        # setup the data path
        #datafile = "FCPO_6_years_NUS.xlsx"
        data = bt.feeds.GenericCSVData(
            dataname=datalist[i][0],
            fromdate= start_test,
            todate= end_test,
            nullvalue=0.0,
            dtformat=('%d/%m/%Y %H:%M'),
            tz = pytz.timezone('Asia/Jakarta'),
            datetime=2,
            high=4,
            low=5,
            open=3,
            close=6,
            volume=7
        )
        # Add the Data Feed to Cerebro
        cerebro.adddata(data, name=datalist[i][1])
        
        #print ("Added data source {}".format(datalist[i][0]))
        #if testmode:
        return 

In [ ]:
def setup_test_data(bt, cerebro):
    import os
    modpath = os.getcwd() + "\\Data\\"
    datapath = os.path.join(modpath, "orcl-1995-2014.txt")
    
    # Create a Data Feed
    data = bt.feeds.YahooFinanceCSVData(
        dataname=datapath,
        # Do not pass values before this date
        fromdate=datetime.datetime(2000, 1, 1),
        # Do not pass values before this date
        todate=datetime.datetime(2000, 12, 31),
        # Do not pass values after this date
        reverse=False)

    # Add the Data Feed to Cerebro
    cerebro.adddata(data)

In [ ]:
def setup_platform(cerebro):
    ''' Setup the rules of engagement'''
    # Set our desired cash start, using MYR
    cerebro.broker.setcash(10000.0)

    # Set the commission - max of 0.2% or RM30
    comminfo = CommInfo_Fut_Perc_Mult(
        commission=0.002#,  # 0.2%
        #mult=1,
    #    margin = 0
    )

    cerebro.broker.addcommissioninfo(comminfo)
    #cerebro.broker.setcommission(commission=0.001)

    
    # Add a FixedSize sizer according to the stake
    cerebro.addsizer(bt.sizers.FixedSize, stake=1)

In [ ]:
class CommInfo_Fut_Perc_Mult(bt.CommInfoBase):
    #params = (
    #  ('stocklike', False)#,  # Futures
      #('commtype', bt.CommInfoBase.COMM_PERC),  # Apply % Commission
      # ('percabs', False),  # pass perc as xx% which is the default
    #)

    def _getcommission(self, size, price, pseudoexec):
        return max(30, size * price * self.p.commission * self.p.mult)

In [ ]:
def setup_plot():
    import warnings
    warnings.filterwarnings('ignore')
    import matplotlib.pylab as pylab
    pylab.rcParams['figure.figsize'] = 45, 30  # that's default image size for this interactive session
    pylab.rcParams['font.family'] = 'sans-serif'
    pylab.rcParams['font.sans-serif'] = ['Bitstream Vera Sans']
    pylab.rcParams['font.serif'] = ['Bitstream Vera Sans']
    pylab.rcParams["font.size"] = "200"

    #plotter = Plotter()
    #back_tester.plot(plotter=plotter)

    #plor the result
    #plt.figure(figsize=(15,20))
    #cerebro.plot(style='candlestick', iplot=False)
    #imagefile = "test_strategy.png"
    #modpath = os.getcwd() + "\\"
    ##datapath = os.path.join(modpath, imagefile)
    #pylab.savefig(datapath)


### Helper functions can be found here

In [ ]:
# https://stackoverflow.com/questions/9489078/how-to-split-a-huge-csv-file-based-on-content-of-first-column
import csv
from itertools import groupby

def split_excel_to_csv():
    datafile = 'FCPO_6_years_NUS.csv'
    modpath = os.getcwd() + "\\Data\\"
    datapath = os.path.join(modpath, datafile)
    csv_header = 'Date,Time,DateTime,Open,High,Low,Close,Volume'
    reader = csv.reader(open(datapath))
    next(reader)
    for key, rows in groupby(reader,lambda row: row[0]):
        if key[-1] in ["1","2","3"]:
            outputpath = os.path.join(modpath+"//Test//","{}.csv".format(key))
        else:
            outputpath = os.path.join(modpath+"//Prod//","{}.csv".format(key))
        with open(outputpath, "w") as output:
            output.write(csv_header + "\n")
            for row in rows:
                row.pop(0) # remove the key
                output.write(",".join(row) + "\n")
                
#split_excel_to_csv()

### Setting up Custom indicators
Besides the standard simple moving average, we also need to setup:
1. Adaptive moving average
1. Typical Price moving average
1. Triangular moving average

In [ ]:
import backtrader as bt
class MyStochastic2(bt.Indicator):
    lines = ('k', 'd', 'mystoc',)
    # manually counted period
    # 14 for the fast moving k
    # 3 for the slow moving d
    # No extra for the previous k (-1) is needed because
    # already buffers more than the 1 period lookback
    # If we were doing d - d(-1), there is nothing making
    # sure k(-1) is being buffered and an extra 1 would be needed
    params = (
        ('k_period', 14),  # lookback period for highest/lowest
        ('d_period', 3),  # smoothing period for d with the SMA
    )

    def __init__(self):
        self.addminperiod(self.p.k_period + self.p.d_period)

    def next(self):
        # Get enough data points to calculate k and do it
        d = self.data.get(size=self.p.k_period)
        hi = max(d)
        lo = min(d)
        self.lines.k[0] = k0 = (self.data[0] - lo) / (hi - lo)
        # Get enough ks to calculate the SMA of k. Assign to d
        last_ks = self.l.k.get(size=self.p.d_period)
        self.lines.d[0] = sum(last_ks) / self.p.d_period
        # Now calculate mystoc
        self.lines.mystoc[0] = abs(k0 - self.l.k[-1]) / 2.0

### Defining strategies here

In [ ]:
#https://www.backtrader.com/docu/talib/talib.html?highlight=moving%20average#moving-averages-and-ma-type
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)

import argparse
import datetime

import backtrader as bt
class TALibStrategy(bt.Strategy):
    params = (('ind', 'sma'), ('doji', True),)

    INDS = ['sma', 'ema', 'stoc', 'rsi', 'macd', 'bollinger', 'aroon',
            'ultimate', 'trix', 'kama', 'adxr', 'dema', 'ppo', 'tema',
            'roc', 'williamsr']

    def __init__(self):
        if self.p.doji:
            bt.talib.CDLDOJI(self.data.open, self.data.high,
                             self.data.low, self.data.close)

        if self.p.ind == 'sma':
            bt.talib.SMA(self.data.close, timeperiod=25, plotname='TA_SMA')
            bt.indicators.SMA(self.data, period=25)
        elif self.p.ind == 'ema':
            bt.talib.EMA(timeperiod=25, plotname='TA_SMA')
            bt.indicators.EMA(period=25)
        elif self.p.ind == 'stoc':
            bt.talib.STOCH(self.data.high, self.data.low, self.data.close,
                           fastk_period=14, slowk_period=3, slowd_period=3,
                           plotname='TA_STOCH')

            bt.indicators.Stochastic(self.data)

        elif self.p.ind == 'macd':
            bt.talib.MACD(self.data, plotname='TA_MACD')
            bt.indicators.MACD(self.data)
            bt.indicators.MACDHisto(self.data)
        elif self.p.ind == 'bollinger':
            bt.talib.BBANDS(self.data, timeperiod=25,
                            plotname='TA_BBANDS')
            bt.indicators.BollingerBands(self.data, period=25)

        elif self.p.ind == 'rsi':
            bt.talib.RSI(self.data, plotname='TA_RSI')
            bt.indicators.RSI(self.data)

        elif self.p.ind == 'aroon':
            bt.talib.AROON(self.data.high, self.data.low, plotname='TA_AROON')
            bt.indicators.AroonIndicator(self.data)

        elif self.p.ind == 'ultimate':
            bt.talib.ULTOSC(self.data.high, self.data.low, self.data.close,
                            plotname='TA_ULTOSC')
            bt.indicators.UltimateOscillator(self.data)

        elif self.p.ind == 'trix':
            bt.talib.TRIX(self.data, timeperiod=25,  plotname='TA_TRIX')
            bt.indicators.Trix(self.data, period=25)

        elif self.p.ind == 'adxr':
            bt.talib.ADXR(self.data.high, self.data.low, self.data.close,
                          plotname='TA_ADXR')
            bt.indicators.ADXR(self.data)

        elif self.p.ind == 'kama':
            bt.talib.KAMA(self.data, timeperiod=25, plotname='TA_KAMA')
            bt.indicators.KAMA(self.data, period=25)
        
        elif self.p.ind == 'trima':
            bt.talib.TRIMA(self.data, timeperiod=25, plotname='TA_TRIMA')
            #bt.indicators.TRIMA(self.data, period=25)
            
        elif self.p.ind == 'typprice':
            bt.talib.TYPPRICE(self.data, timeperiod=25, plotname='TA_TYPPRICE')
            #bt.indicators.TRIMA(self.data, period=25)

        elif self.p.ind == 'dema':
            bt.talib.DEMA(self.data, timeperiod=25, plotname='TA_DEMA')
            bt.indicators.DEMA(self.data, period=25)

        elif self.p.ind == 'ppo':
            bt.talib.PPO(self.data, plotname='TA_PPO')
            bt.indicators.PPO(self.data, _movav=bt.indicators.SMA)

        elif self.p.ind == 'tema':
            bt.talib.TEMA(self.data, timeperiod=25, plotname='TA_TEMA')
            bt.indicators.TEMA(self.data, period=25)

        elif self.p.ind == 'roc':
            bt.talib.ROC(self.data, timeperiod=12, plotname='TA_ROC')
            bt.talib.ROCP(self.data, timeperiod=12, plotname='TA_ROCP')
            bt.talib.ROCR(self.data, timeperiod=12, plotname='TA_ROCR')
            bt.talib.ROCR100(self.data, timeperiod=12, plotname='TA_ROCR100')
            bt.indicators.ROC(self.data, period=12)
            bt.indicators.Momentum(self.data, period=12)
            bt.indicators.MomentumOscillator(self.data, period=12)

        elif self.p.ind == 'williamsr':
            bt.talib.WILLR(self.data.high, self.data.low, self.data.close,
                           plotname='TA_WILLR')
            bt.indicators.WilliamsR(self.data)

In [ ]:
class St(bt.Strategy):
    params = dict(
        enter=[1, 3, 4],  # data ids are 1 based
        hold=[7, 10, 15],  # data ids are 1 based
        usebracket=True,
        rawbracket=True,
        pentry=0.015,
        plimits=0.03,
        valid=10,
    )

    def notify_order(self, order):
        if order.status == order.Submitted:
            return

        dt, dn = self.datetime.date(), order.data._name
        print('{} {} Order {} Status {}'.format(
            dt, dn, order.ref, order.getstatusname())
        )

        whichord = ['main', 'stop', 'limit', 'close']
        if not order.alive():  # not alive - nullify
            dorders = self.o[order.data]
            idx = dorders.index(order)
            dorders[idx] = None
            print('-- No longer alive {} Ref'.format(whichord[idx]))

            if all(x is None for x in dorders):
                dorders[:] = []  # empty list - New orders allowed

    def __init__(self):
        self.o = dict()  # orders per data (main, stop, limit, manual-close)
        self.holding = dict()  # holding periods per data
        print ("Starting strategy St:")

    def next(self):
        for i, d in enumerate(self.datas):
            dt, dn = self.datetime.date(), d._name
            pos = self.getposition(d).size
            print('{} {} Position {}'.format(dt, dn, pos))

            if not pos and not self.o.get(d, None):  # no market / no orders
                if dt.weekday() == self.p.enter[i]:
                    if not self.p.usebracket:
                        self.o[d] = [self.buy(data=d)]
                        print('{} {} Buy {}'.format(dt, dn, self.o[d][0].ref))

                    else:
                        p = d.close[0] * (1.0 - self.p.pentry)
                        pstp = p * (1.0 - self.p.plimits)
                        plmt = p * (1.0 + self.p.plimits)
                        valid = datetime.timedelta(self.p.valid)

                        if self.p.rawbracket:
                            o1 = self.buy(data=d, exectype=bt.Order.Limit,
                                          price=p, valid=valid, transmit=False)

                            o2 = self.sell(data=d, exectype=bt.Order.Stop,
                                           price=pstp, size=o1.size,
                                           transmit=False, parent=o1)

                            o3 = self.sell(data=d, exectype=bt.Order.Limit,
                                           price=plmt, size=o1.size,
                                           transmit=True, parent=o1)

                            self.o[d] = [o1, o2, o3]

                        else:
                            self.o[d] = self.buy_bracket(
                                data=d, price=p, stopprice=pstp,
                                limitprice=plmt, oargs=dict(valid=valid))

                        print('{} {} Main {} Stp {} Lmt {}'.format(
                            dt, dn, *(x.ref for x in self.o[d])))

                    self.holding[d] = 0

            elif pos:  # exiting can also happen after a number of days
                self.holding[d] += 1
                if self.holding[d] >= self.p.hold[i]:
                    o = self.close(data=d)
                    self.o[d].append(o)  # manual order to list of orders
                    print('{} {} Manual Close {}'.format(dt, dn, o.ref))
                    if self.p.usebracket:
                        self.cancel(self.o[d][1])  # cancel stop side
                        print('{} {} Cancel {}'.format(dt, dn, self.o[d][1]))

In [ ]:
### https://ntguardian.wordpress.com/2017/06/12/getting-started-with-backtrader/
class SMAC(bt.Strategy):
    """A simple moving average crossover strategy; crossing of a fast and slow moving average generates buy/sell
       signals"""
    params = {"ind":"sma",
              "fast": 20, 
              "slow": 50,                  # The windows for both fast and slow moving averages
              "optim": False, 
              "optim_fs": (20, 50)}    # Used for optimization; equivalent of fast and slow, but a tuple
                                                       # The first number in the tuple is the fast MA's window, the
                                                       # second the slow MA's window
 
    def log(self, txt, dt=None):
        ''' Logging function fot this strategy'''
        dt = dt or self.datas[dataIdx].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        """Initialize the strategy"""
 
        self.fastma = dict()
        self.slowma = dict()
        self.regime = dict()
        
        # To keep track of pending orders and buy price/commission
        self.order = None
        self.buyprice = None
        self.buycomm = None
 
        self._addobserver(True, bt.observers.BuySell)    # CAUTION: Abuse of the method, I will change this in future code (see: https://community.backtrader.com/topic/473/plotting-just-the-account-s-value/4)
 
        if self.params.optim:    # Use a tuple during optimization
            self.params.fast, self.params.slow = self.params.optim_fs    # fast and slow replaced by tuple's contents
 
        if self.params.fast > self.params.slow:
            raise ValueError(
                "A SMAC strategy cannot have the fast moving average's window be " + \
                 "greater than the slow moving average window.")
 
        for d in self.getdatanames():
 
            # The moving averages
            self.fastma[d] = bt.indicators.SimpleMovingAverage(self.getdatabyname(d),      # The symbol for the moving average
                                                       period=self.params.fast,    # Fast moving average
                                                       plotname="FastMA: " + d)
            self.slowma[d] = bt.indicators.SimpleMovingAverage(self.getdatabyname(d),      # The symbol for the moving average
                                                       period=self.params.slow,    # Slow moving average
                                                       plotname="SlowMA: " + d)
 
            # Get the regime
            self.regime[d] = self.fastma[d] - self.slowma[d]    # Positive when bullish
 
    def next(self):
        """Define what will be done in a single step, including creating and closing trades"""
        for i, d in enumerate(self.datas):
            dt, dn = self.datetime.date(), d._name
            pos = self.getposition(d).size
            if not pos:
            
        #for d in self.getdatanames():    # Looping through all symbols
            #pos = self.getpositionbyname(d).size or 0
            #if pos == 0:    # Are we out of the market?
                # Consider the possibility of entrance
                # Notice the indexing; [0] always means the present bar, and [-1] the bar immediately preceding
                # Thus, the condition below translates to: "If today the regime is bullish (greater than
                # 0) and yesterday the regime was not bullish"
                if self.regime[dn][0] > 0 and self.regime[dn][-1] <= 0:    # A buy signal
                    # BUY, BUY, BUY!!! (with default parameters)
                    self.log('%s BUY CREATE, %.2f with volume %.2f' % (dn, d.high[0], d.volume[0]), dt=d.datetime.date(0))

                    # Keep track of the created order to avoid a 2nd order
                    self.order = self.buy(data=d, price=d.high[0])
                    
            else:    # We have an open position
                if self.regime[dn][0] <= 0 and self.regime[dn][-1] > 0:    # A sell signal
                    # SELL, SELL, SELL!!! (with all possible default parameters)
                    self.log('%s SELL CREATE, %.2f with volume %.2f' % (dn, d.low[0], d.volume[0]), dt=d.datetime.date(0))

                    # Keep track of the created order to avoid a 2nd order
                    self.order = self.sell(data=d, price=d.low[0])
                    
    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log('BUY EXECUTED, Size: %.2f, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.size,
                     order.executed.price,
                     order.executed.value,
                     order.executed.comm))

                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            elif order.issell():
                self.log('SELL EXECUTED, Size: %.2f, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.size,
                          order.executed.price,
                          order.executed.value,
                          order.executed.comm))

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        # Write down: no pending order
        self.order = None
    
    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))

In [ ]:
dataIdx = 0

# Create a Stratey
class TestStrategy(bt.Strategy):
    params = (
        ('maperiod', 15),
    )

    def log(self, txt, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.datas[dataIdx].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[dataIdx].close
        print ("Initiated")

        # To keep track of pending orders and buy price/commission
        self.order = None
        self.buyprice = None
        self.buycomm = None

        # Add a MovingAverageSimple indicator
        self.sma = bt.indicators.SimpleMovingAverage(
            self.datas[dataIdx], period=self.params.maperiod)


    def next(self):
        # Simply log the closing price of the series from the reference
        self.log('Close, %.2f' % self.dataclose[0])

        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return

        # Check if we are in the market
        if not self.position:

            # Not yet ... we MIGHT BUY if ...
            if self.dataclose[0] > self.sma[0]:

                # BUY, BUY, BUY!!! (with all possible default parameters)
                self.log('BUY CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.buy()

        else:

            if self.dataclose[0] < self.sma[0]:
                # SELL, SELL, SELL!!! (with all possible default parameters)
                self.log('SELL CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.sell()
                
    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log('BUY EXECUTED, Size: %.2f, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.size,
                     order.executed.price,
                     order.executed.value,
                     order.executed.comm))

                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            elif order.issell():
                self.log('SELL EXECUTED, Size: %.2f, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.size,
                          order.executed.price,
                          order.executed.value,
                          order.executed.comm))

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        # Write down: no pending order
        self.order = None
    
    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))

In [ ]:
class TypicalPriceMovingAverage(bt.Indicator):
    lines = ('tpma',)
    params = (('period', 20),)

    def __init__(self):
        self.addminperiod(self.params.period)

    def next(self):
        dataline = self.data.get(size=self.p.period)
        datasum = max(dataline) + min(dataline) + dataline[-1]
        self.lines.tpma[0] = datasum/3

In [ ]:
### https://ntguardian.wordpress.com/2017/06/12/getting-started-with-backtrader/
class AMAC(bt.Strategy):
    """A simple moving average crossover strategy; crossing of a fast and slow moving average generates buy/sell
       signals"""
    params = {"fast_ind":"sma",
              "slow_ind":"sma",
              "fast": 1, 
              "slow": 10,                  # The windows for both fast and slow moving averages
              "optim": False, 
              "optim_fs": (20, 50)}    # Used for optimization; equivalent of fast and slow, but a tuple
                                                       # The first number in the tuple is the fast MA's window, the
                                                       # second the slow MA's window
 
    def setIndicator(self, ma_type, d):
        plottitle = "SlowMA: "
        period_val = self.params.slow
        ind_type = self.p.slow_ind
        if ma_type == "fast":
            plottitle = "FastMA: "
            period_val = self.params.fast
            ind_type = self.p.fast_ind
            
        datafeed = self.getdatabyname(d)
        
        print ("ind:{}, ma:{}, period:{}".format(ind_type, ma_type, period_val))
        ma = None
        if ind_type == 'sma':
            ma = bt.indicators.SMA(datafeed,      # The symbol for the moving average
                                   period=period_val,    # Fast moving average
                                   plotname=plottitle + d)

        elif ind_type == "ama":
            ma = bt.indicators.KAMA(datafeed,      # The symbol for the moving average
                                    period=period_val,    # Fast moving average
                                    plotname=plottitle + d)

        elif ind_type == "tma":
            ma = bt.talib.TRIMA(datafeed,      # The symbol for the moving average
                               timeperiod=period_val,    # Fast moving average
                               plotname=plottitle + d)
        elif ind_type == "tpma":
            ma = TypicalPriceMovingAverage(datafeed,      # The symbol for the moving average
                                           period=period_val,    # Fast moving average
                                           plotname=plottitle + d)
        return ma

    def log(self, txt, dt=None):
        ''' Logging function fot this strategy'''
        dt = dt or self.datas[dataIdx].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        """Initialize the strategy"""
 
        self.fastma = dict()
        self.slowma = dict()
        self.regime = dict()
        
        # To keep track of pending orders and buy price/commission
        self.order = None
        self.buyprice = None
        self.buycomm = None
 
        self._addobserver(True, bt.observers.BuySell)    # CAUTION: Abuse of the method, I will change this in future code (see: https://community.backtrader.com/topic/473/plotting-just-the-account-s-value/4)
 
        if self.params.optim:    # Use a tuple during optimization
            self.params.fast, self.params.slow = self.params.optim_fs    # fast and slow replaced by tuple's contents
 
        if self.params.fast > self.params.slow:
            raise ValueError(
                "A SMAC strategy cannot have the fast moving average's window be " + \
                 "greater than the slow moving average window.")
 
        for d in self.getdatanames():
 
            # The moving averages
            self.fastma[d] = self.setIndicator("fast",d)
            self.slowma[d] = self.setIndicator("slow",d)

            #bt.talib.KAMA(self.data, timeperiod=25, plotname='TA_KAMA')
            #bt.indicators.KAMA(self.data, period=25)
            
            #self.log("Testing %.2f", bt.talib.TYPPRICE(10))
 
            # Get the regime
            self.regime[d] = self.fastma[d] - self.slowma[d]    # Positive when bullish
            #self.log("Regime: {}".format(self.regime[d][0]))
 
    def next(self):
        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return
        
        """Define what will be done in a single step, including creating and closing trades"""
        for i, d in enumerate(self.datas):
            dt, dn = self.datetime.date(), d._name
            pos = self.getposition(d).size
            #print ("Position: {}".format(pos))
            #if not pos:
            
            
            
        #for d in self.getdatanames():    # Looping through all symbols
            #pos = self.getpositionbyname(d).size or 0
            if pos == 0:    # Are we out of the market?
                # Consider the possibility of entrance
                # Notice the indexing; [0] always means the present bar, and [-1] the bar immediately preceding
                # Thus, the condition below translates to: "If today the regime is bullish (greater than
                # 0) and yesterday the regime was not bullish"
                if self.regime[dn][0] > 0 and self.regime[dn][-1] <= 0:    # A buy signal
                    # BUY, BUY, BUY!!! (with default parameters)
                    self.log('%s BUY CREATE, %.2f with volume %.2f' % (dn, d.high[0], d.volume[0]), dt=d.datetime.date(0))

                    # Keep track of the created order to avoid a 2nd order
                    self.order = self.buy(data=d, price=d.high[0])
                    
            else:    # We have an open position
                if self.regime[dn][0] <= 0 and self.regime[dn][-1] > 0:    # A sell signal
                    # SELL, SELL, SELL!!! (with all possible default parameters)
                    self.log('%s SELL CREATE, %.2f with volume %.2f' % (dn, d.low[0], d.volume[0]), dt=d.datetime.date(0))

                    # Keep track of the created order to avoid a 2nd order
                    self.order = self.sell(data=d, price=d.low[0])
                    
    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log('BUY EXECUTED, Size: %.2f, Price: %.2f, Cost: %.2f, Comm %.2f, Portfolio Value: %.2f, Cash: %.2f' %
                    (order.executed.size,
                     order.executed.price,
                     order.executed.value,
                     order.executed.comm,
                     self.stats.broker.value[0],
                     self.stats.broker.cash[0]))

                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            elif order.issell():
                self.log('SELL EXECUTED, Size: %.2f, Price: %.2f, Cost: %.2f, Comm %.2f, Portfolio Value: %.2f, Cash: %.2f' %
                         (order.executed.size,
                          order.executed.price,
                          order.executed.value,
                          order.executed.comm,
                         self.stats.broker.value[0],
                         self.stats.broker.cash[0]))

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        # Write down: no pending order
        self.order = None
    
    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))

In [ ]:
def return_ga():
    return 0

### And the paper chase begins here

In [ ]:
#from report import Cerebro

if __name__ == '__main__':
    # Create a cerebro entity
    cerebro = bt.Cerebro()

    # Add a strategy
    cerebro.addstrategy(AMAC, fast_ind="tpma", slow_ind="tpma", fast=10, slow=20)
    #cerebro.addstrategy(TestStrategy)
    #cerebro.addstrategy(TALibStrategy, ind="0", doji=5)

    # Create a Data Feed
    setup_multiple_data(bt, cerebro)
    #setup_test_data(bt, cerebro)

    setup_platform(cerebro)

    # Print out the starting conditions
    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

    # Run over everything
    cerebro.run()

    # Print out the final result
    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
    
    # Plot the result
    #setup_plot()
    cerebro.plot(style='candlestick', iplot=True)

In [ ]:
#pylab.rcParams['figure.figsize'] = 12, 8  # that's default image size for this interactive session
imagefile = "test_strategy.png"
modpath = os.getcwd() + "\\"
datapath = os.path.join(modpath, imagefile)
#pylab.savefig(datapath)
print (datapath)
cerebro.plot(style='candlestick', iplot=False, ytight=True, subplot=False, path=datapath)

In [ ]:
from report import Cerebro


In [ ]:
import backtrader.plot as plt


class Plotter(plt.Plot):

    def __init__(self):
        super().__init__(volup='#60cc73')  # custom color for volume up bars 

    def show(self):
        mng = self.mpyplot.get_current_fig_manager()
        mng.window.state('zoomed')
        self.mpyplot.show()

### Building the fuzzy extents

In [ ]:
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl

# New Antecedent/Consequent objects hold universe variables and membership
# functions
quality = ctrl.Antecedent(np.arange(0, 11, 1), 'quality')
service = ctrl.Antecedent(np.arange(0, 11, 1), 'service')
tip = ctrl.Consequent(np.arange(-30, 30, 1), 'tip')

# Auto-membership function population is possible with .automf(3, 5, or 7)
quality.automf(3)
service.automf(3)

# Custom membership functions can be built interactively with a familiar,
# Pythonic API
#tip['low'] = fuzz.trimf(tip.universe, [0, 0, 13])
#tip['medium'] = fuzz.trimf(tip.universe, [0, 13, 25])
#tip['high'] = fuzz.trimf(tip.universe, [13, 25, 25])
arr = []
sigma = 2.5
tip["EL"] = fuzz.gaussmf(tip.universe, -30, sigma)
tip["VL"] = fuzz.gaussmf(tip.universe, -20, sigma)
tip["L"] = fuzz.gaussmf(tip.universe, -10, sigma)
tip["M"] = fuzz.gaussmf(tip.universe, 0, sigma)
tip["H"] = fuzz.gaussmf(tip.universe, 10, sigma)
tip["VH"] = fuzz.gaussmf(tip.universe, 20, sigma)
tip["EH"] = fuzz.gaussmf(tip.universe, 30, sigma)

tip.view()

In [1]:
#!/usr/bin/env python
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl

# New Antecedent/Consequent objects hold universe variables and membership functions

tip = ctrl.Antecedent(np.arange(-30, 30, 0.1), 'tip')
null= ctrl.Consequent(np.arange(0, 30, 1), 'null')

# Auto-membership function population is possible with .automf(3, 5, or 7)
null.automf(3)

#Custom membership functions can be built interactively with a familiar, Pythonic API
arr = []
sigma = 2.5
tip["EL"] = fuzz.gaussmf(tip.universe, -30, sigma)
tip["VL"] = fuzz.gaussmf(tip.universe, -20, sigma)
tip["L"] = fuzz.gaussmf(tip.universe, -10, sigma)
tip["M"] = fuzz.gaussmf(tip.universe, 0, sigma)
tip["H"] = fuzz.gaussmf(tip.universe, 10, sigma)
tip["VH"] = fuzz.gaussmf(tip.universe, 20, sigma)
tip["EH"] = fuzz.gaussmf(tip.universe, 30, sigma)

#tip.view()
'''
and after creating this, I would like to be able to get the value back based on each fuzzy segment that I've created as follows, tip["EH"].input = 5 and based on the bell curve created, I would like to get back the membership function value as per the plot. I've looked through the document but was'nt able to get it. Please help! Thanks
'''

rule =   ctrl.Rule(tip["EH"],null['good'])


ctrl_sys = ctrl.ControlSystem([rule])

ctrl_instance = ctrl.ControlSystemSimulation(ctrl_sys)

ctrl_instance.input['tip'] = 23

ctrl_instance.compute()

tip.view(sim=ctrl_instance)

print (ctrl_instance.output['null'])

#raw_input()

21.64291029863274


C:\Users\mela_\Anaconda3\lib\site-packages\matplotlib\figure.py:459: UserWarning: matplotlib is currently using a non-GUI backend, so cannot show the figure
  "matplotlib is currently using a non-GUI backend, "


In [2]:
ctrl_instance.print_state()


 Antecedents 
Antecedent: tip                     = 23
  - EL                              : 2.5429863364472064e-98
  - VL                              : 5.74328326744169e-65
  - L                               : 1.459703793211397e-38
  - M                               : 4.175010055852897e-19
  - H                               : 1.343812277631746e-06
  - VH                              : 0.48675225595997285
  - EH                              : 0.01984109474437108

 Rules 
RULE #0:
  IF tip[EH] THEN null[good]
	AND aggregation function : fmin
	OR aggregation function  : fmax

  Aggregation (IF-clause):
  - tip[EH]                                                : 0.01984109474437108
                                                   tip[EH] = 0.01984109474437108
  Activation (THEN-clause):
                                                null[good] : 0.01984109474437108


 Intermediaries and Conquests 
Consequent: null                     = 21.64291029863274
  poor:


AttributeError: 'function' object has no attribute 'func_name'

In [ ]:
# Set inputs
tip.input['quality'] = 8
tip.input['service'] = 7

# Run the system
tip.compute()

# Extract the antecedent you want (just one of multiple ways)
ants = [i for i in tip.ctrl.antecedents]

ant = ants[0]  # Select the antecedent you want

# Some variation on this to yield the membership values
print(['For term \'{0}\' membership is {1}'.format(label, term.membership_value[tip_sim]) 
       for (label, term) in ant.terms.iteritems()])

In [ ]:
import random
import numpy as np
data= np.array(random.sample(range(-100,100),20))
sigma = np.std(data)
print (sigma/7)

# break the list into those t

In [ ]:
# You can see how these look with .view()
#quality['average'].view()
tip.view()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import skfuzzy as fuzz


# Generate trapezoidal membership function on range [0, 1]
x = np.arange(0, 5.05, 0.1)
mfx = fuzz.trapmf(x, [2, 2.5, 3, 4.5])

# Defuzzify this membership function five ways
defuzz_centroid = fuzz.defuzz(x, mfx, 'centroid')  # Same as skfuzzy.centroid
defuzz_bisector = fuzz.defuzz(x, mfx, 'bisector')
defuzz_mom = fuzz.defuzz(x, mfx, 'mom')
defuzz_som = fuzz.defuzz(x, mfx, 'som')
defuzz_lom = fuzz.defuzz(x, mfx, 'lom')

# Collect info for vertical lines
labels = ['centroid', 'bisector', 'mean of maximum', 'min of maximum',
          'max of maximum']
xvals = [defuzz_centroid,
         defuzz_bisector,
         defuzz_mom,
         defuzz_som,
         defuzz_lom]
colors = ['r', 'b', 'g', 'c', 'm']
ymax = [fuzz.interp_membership(x, mfx, i) for i in xvals]
print (ymax)

# Display and compare defuzzification results against membership function
plt.figure(figsize=(8, 5))

plt.plot(x, mfx, 'k')
for xv, y, label, color in zip(xvals, ymax, labels, colors):
    plt.vlines(xv, 0, y, label=label, color=color)
plt.ylabel('Fuzzy membership')
plt.xlabel('Universe variable (arb)')
plt.ylim(-0.1, 1.1)
plt.legend(loc=2)

plt.show()